In [1]:
import pandas as pd 
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
import torch

In [2]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
dropout = 0.2
# ------------

In [3]:
torch.manual_seed(1337)

In [ ]:
PATH  ="/kaggle/input/shakespeare6/shakespeare.txt"

with open(PATH, 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print("Dataset character length: ", len(text))

Dataset character length:  13593923


In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [7]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [8]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2,-1) * C**-0.6
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        out = wei @ v
        return out

In [11]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [12]:
import torch.nn as nn
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(n_embd, 4*n_embd),
                                 nn.ReLU(),
                                 nn.Linear(4* n_embd, n_embd),
                                 nn.Dropout(dropout),
                                )
    def forward(self,x):
        return self.net(x)

In [13]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self,x):
        x = x+self.sa(self.ln1(x))
        x = x+self.ffwd(self.ln2(x))
        return x

In [14]:
import torch.nn as nn
import torch.optim as optim
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            nn.LayerNorm(n_embd),
        )
        self.lm_head = nn.Linear(n_embd, vocab_size)
    def forward(self, idx, targets=None):
        B,T=idx.shape
        # idx and targets are both (B,T) tensor of integers
        token_emb = self.token_embedding_table(idx) # (B,T,C)
        # logits = self.linear(token_emb) # (B,T,vocab_size)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = token_emb+pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size of the tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [15]:
model = BigramLanguageModel()
m = model.to(device)

In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [17]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % 200 == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    if max_iters >= 10000:
        learning_rate = 1e-3
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.4981, val loss 4.4981
step 200: train loss 2.4444, val loss 2.4589
step 400: train loss 2.3489, val loss 2.3627
step 600: train loss 2.0764, val loss 2.0981
step 800: train loss 1.8485, val loss 1.8754
step 1000: train loss 1.6993, val loss 1.7279
step 1200: train loss 1.5810, val loss 1.6145
step 1400: train loss 1.4948, val loss 1.5328
step 1600: train loss 1.4258, val loss 1.4620
step 1800: train loss 1.3753, val loss 1.4155
step 2000: train loss 1.3361, val loss 1.3793
step 2200: train loss 1.3055, val loss 1.3498
step 2400: train loss 1.2887, val loss 1.3311
step 2600: train loss 1.2656, val loss 1.3066
step 2800: train loss 1.2422, val loss 1.2861
step 3000: train loss 1.2296, val loss 1.2737
step 3200: train loss 1.2135, val loss 1.2600
step 3400: train loss 1.2084, val loss 1.2526
step 3600: train loss 1.1937, val loss 1.2424
step 3800: train loss 1.1807, val loss 1.2319
step 4000: train loss 1.1769, val loss 1.2203
step 4200: train loss 1.1685, val loss 1.

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


that hormisions at the if, subt past the celestial cure hundred to them
throefore, only the dleads. For head may adart, knowing, O monarch, succests who
lossome shingled and the infant viok, said, 'King Duryodhana thereat
mighty that hand occapable of them and excited and persons to foes! Those
heaven, O tiger enorto wealth to thee with the ent
of thy broad-king, indeed, causpiciousne kings, by
ply. What is so also the permissibly filler of all speaking the just,
before our life.' Brahmana cembo


In [ ]:
torch.save(model.state_dict(), "gpt_weights.pth")